## Read before proceed

0. Most of the codes here are copied from [tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html). I don`t think these codes comply with the currently accepted standards. **So treat them carefully**
1. This `.ipynb` file should be placed outside of `kaggleData` file
2. Change `file_path` if running in a local machine with gpu
3. The `n_words` (vocab size) of `Lang` class affects the dimensions of Encoder, but our hw requires a random split of 90% for training, hence such split changes `n_words` every time you run this file for training. So i set `random_state = 1234` at `sklearn.train_test_split` to make training reproducible. 
4. Do not change `random_state` if you are using the existed weights `encoder_weights.pth` `decoder_weights.pth`. If you change `random_state`, you will have to re-train yourself.

In [18]:
# from __future__ import unicode_literals, print_function, division
# from io import open
# ======= features from python 2.x, no need here ========
import unicodedata
import re
import random
import torch
import numpy as np

import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from sklearn.model_selection import train_test_split


# ========================= PATH ================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device.type == 'cpu':
    file_path = './kaggleData/data/'
else:
    file_path = '/kaggle/input/eng-cmn/'


In [19]:
# encoding words

SOS_token = 0
EOS_token = 1
UNK_token = 2

# old Land: count based
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {"<UNK>": UNK_token}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2: "<UNK>"}
        self.n_words = 3  # Count SOS and EOS + UNK

    def addSentence(self, sentence):

        for word in sentence.split(' '):
            self.addWord(word)


    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1

        else:
            self.word2count[word] += 1


# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# deal with english letters: lower case + strip all punctuation
def normalize_english(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

# deal with chinese words: strip all punctuation
def normalize_chinese(s):
    s = s.strip()
    s = ' '.join(list(s)) 
    s = re.sub(r'\s+', ' ', s)
    return s.strip()


In [20]:
# read data (cn2eng)
def read_langs(lang1, lang2, file_path = file_path, test_size = 0.1, reverse=False):
    # lines = open('data/cmn.txt', encoding='utf-8').read().split('\n')
    print("Reading lines...")

    # Read the file and split into lines
    lines = open(file_path + '%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')
    print(file_path + '%s-%s.txt' % (lang1, lang2))
    pairs = []
    for line in lines:
        parts = line.split('\t')
        if len(parts) < 2: continue
        en = normalize_english(parts[0].strip())
        zh = normalize_chinese(parts[1].strip())
        pairs.append([zh, en] if reverse else [en, zh])
    input_lang = Lang(lang2) if reverse else Lang(lang1)
    output_lang = Lang(lang1) if reverse else Lang(lang2)



    # ================== split data to train/test set ==================
    
    # random_state for reproduction, otherwise saved weights cannot be loaded properly
    pairs, pairs_test = train_test_split(pairs, test_size=test_size, shuffle=True, random_state = 1234)
    return input_lang, output_lang, pairs, pairs_test
    # return input_lang, output_lang, pairs


In [ ]:
# filter data, extract short sentence

MAX_LENGTH = 45 # 45: the minimal length that can use the whole data set without being trimmed/filtered

# filter out text-pairs that exceed MAX_LENGTH: for test
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH 

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def prepareData(lang1, lang2, file_path ,test_size = 0.1, reverse=False):
    # input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    input_lang, output_lang, pairs, pairs_test = read_langs(lang1, lang2,file_path, test_size , reverse)
    # input_lang, output_lang, pairs = read_langs(lang1, lang2,file_path, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    pairs_test = filterPairs(pairs_test)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs, pairs_test


# ============ test if data are loaded and prepared as expected: =============
input_lang, output_lang, pairs, pairs_test = prepareData('eng', 'cmn',file_path = file_path, test_size = 0.1, reverse=True)
print(random.choice(pairs))
print(input_lang.n_words)



In [22]:
# define encode-decoder rnn

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden
    


class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden
    

# introduce decoder with attn
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

In [23]:
# prepare data from training
def indexesFromSentence(lang, sentence):
    return [lang.word2index.get(word, UNK_token) for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size, test_size):
    # input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
    # pairs: 90% data for train
    # pairs_test: 10% data for test
    input_lang, output_lang, pairs, pairs_test = prepareData('eng', 'cmn', file_path=file_path, test_size = test_size, reverse=True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    # ============== build data loader =======================
    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    # ===============================================
    
    return input_lang, output_lang, train_dataloader, pairs_test

In [24]:
# training misc: define training loop / plot loss curve / print time consumed etc

def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)


import matplotlib.pyplot as plt
# plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.savefig('seq2seq_loss.png')

import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))


def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=10, plot_every=10):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()
    # print(f'plot_every: {plot_every}')

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [25]:
# eval 

def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        # input_tensor = sentence
        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])

    return decoded_words, decoder_attn

# pick n = 20 entries from test set to evaluate
def evaluateRandomly(text_pairs, encoder, decoder,  n=20):
    sentence = []
    for i in range(n):
        pair = random.choice(text_pairs)
        sentence.append(pair[0])
        print('>', pair[0])
        print('=', pair[1])
        print(pair[0])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')
    return sentence
    

In [ ]:
# hyper params
hidden_size = 128 # a.k.a 'd_model': max num of dimension of input tensor the model can deal with
batch_size = 32
max_epochs = 80
test_size = 0.1 # 10% of the data will be used as test set
print_every = 5 # print training info (loss) every 5 epoch
plot_every = 5 # record loss value every 5 epoch

input_lang, output_lang, train_dataloader, pairs_test = get_dataloader(batch_size, test_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

# start training
train(train_dataloader, encoder, decoder, max_epochs, print_every=5, plot_every=5)

In [27]:
# save model
torch.save(encoder.state_dict(), "encoder_weights.pth")
torch.save(decoder.state_dict(), "decoder_weights.pth")

In [ ]:
# reload model from saved file
encoder.load_state_dict(torch.load('encoder_weights.pth', map_location=device))
decoder.load_state_dict(torch.load('decoder_weights.pth', map_location=device))
encoder.eval()
decoder.eval()

In [ ]:
# randomly evalue (translate) n=100 chinese text from test set
sentence = evaluateRandomly(pairs_test, encoder, decoder, n=50) # < ------------ test set

In [31]:
# visualize attention

def showAttention(input_sentence, output_words, attentions):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.cpu().numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(encoder, decoder, input_sentence, input_lang, output_lang)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions[0, :len(output_words), :])

In [ ]:
for sentence_ in sentence[:10]:
    evaluateAndShowAttention(sentence_)